In [1]:
import gradio as gr
import joblib
from bs4 import BeautifulSoup
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
# Load the pre-trained model and vectorizer
clf = joblib.load('reddit_upvote_downvote_model.joblib')
vectorizer = joblib.load('tfidf_vectorizer.joblib')

In [3]:
# Initialize the lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

In [4]:
# Function to preprocess the input message
def clean_text(text):
    text = re.sub(r'http\S+', '', text)
    soup = BeautifulSoup(text, 'html.parser')
    text = soup.get_text()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text).lower()
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    cleaned_text = ' '.join(tokens)
    return cleaned_text

In [5]:
# Function to predict upvote/downvote and display probabilities
def predict(msg):
    preprocessed_msg = clean_text(msg)
    msg_input = vectorizer.transform([preprocessed_msg])
    probabilities = clf.predict_proba(msg_input)

    upvote_probability = probabilities[0][1] * 100
    downvote_probability = probabilities[0][0] * 100

    return f"Prediction: {clf.predict(msg_input)[0]}", f"Upvote Probability: {upvote_probability:.2f}%", f"Downvote Probability: {downvote_probability:.2f}%"


In [6]:
# Gradio Interface
iface = gr.Interface(fn=predict, inputs="text", outputs=["text", "text", "text"])
iface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
